In [1]:
import datajoint as dj
import numpy as np
import time
from tqdm import tqdm

In [2]:
#setting the address and the username
dj.config['database.host'] = '10.28.0.34'
dj.config['database.user'] = 'celiib'
dj.config['database.password'] = 'newceliipass'
dj.config['safemode']=True
dj.config["display.limit"] = 200

ta3p100 = dj.create_virtual_module('ta3p100', 'microns_ta3p100')
schema = dj.schema('microns_ta3p100')


Connecting celiib@10.28.0.34:3306


In [3]:
@schema
class CoarseLabel(dj.Computed):
    definition = """
    # Vertex labels for ta3p100.ProofreadLabel did not correctly match the triangle labels, so these are regenerated from the correct triangle labels.
    -> ta3p100.ProofreadLabel
    ---
    vertices  : longblob # Corrected vertex labels
    triangles : longblob # Same triangle labels as ta3p100.ProofreadLabel
    """
    
    key_source = ta3p100.ProofreadLabel & 'status="complete"'
    
    def make(self, key):
        start = time.time()
        
        print(key["segment_id"])
        labels = (ta3p100.ProofreadLabel & key).fetch1()
        corrected_vertex_labels = np.zeros(labels['vertices'].shape, np.uint8)
        
        mesh = (ta3p100.CleansedMesh & 'decimation_ratio=0.35' & dict(segment_id=key['segment_id'])).fetch1()
        mesh_triangles = mesh['triangles']
        
        vertex_label_dict = dict()
        for i, label in enumerate(labels['triangles']):
            triangle = mesh_triangles[i]
            for node in triangle:
                if node in vertex_label_dict:
                    if vertex_label_dict[node] < label:
                        vertex_label_dict[node] = label
                else:
                    vertex_label_dict[node] = label
                
        for node, label in vertex_label_dict.items():
            corrected_vertex_labels[node] = label
            
        self.insert1(dict(key,
                          vertices=corrected_vertex_labels,
                          triangles=labels['triangles']))
        
        print("Segment {} vertex labels regenerated in: {} seconds.".format(key['segment_id'], time.time() - start))

In [4]:
start = time.time()
CoarseLabel.populate(reserve_jobs=True)
print(time.time() - start)

648518346349474146
Segment 648518346349474146 vertex labels regenerated in: 5.479918003082275 seconds.
648518346349475101
Segment 648518346349475101 vertex labels regenerated in: 2.889734983444214 seconds.
648518346349475132
Segment 648518346349475132 vertex labels regenerated in: 4.250591993331909 seconds.
648518346349475436
Segment 648518346349475436 vertex labels regenerated in: 1.920802116394043 seconds.
648518346349475518
Segment 648518346349475518 vertex labels regenerated in: 5.735827207565308 seconds.
648518346349475523
Segment 648518346349475523 vertex labels regenerated in: 3.2772159576416016 seconds.
648518346349475526
Segment 648518346349475526 vertex labels regenerated in: 3.2885282039642334 seconds.
648518346349475530
Segment 648518346349475530 vertex labels regenerated in: 2.7867298126220703 seconds.
648518346349475536
Segment 648518346349475536 vertex labels regenerated in: 5.415845155715942 seconds.
648518346349476185
Segment 648518346349476185 vertex labels regenerate

Segment 648518346349504955 vertex labels regenerated in: 4.700931787490845 seconds.
648518346349505250
Segment 648518346349505250 vertex labels regenerated in: 6.016232252120972 seconds.
648518346349505592
Segment 648518346349505592 vertex labels regenerated in: 5.238929748535156 seconds.
648518346349505739
Segment 648518346349505739 vertex labels regenerated in: 3.681570053100586 seconds.
648518346349505911
Segment 648518346349505911 vertex labels regenerated in: 4.355692148208618 seconds.
648518346349506159
Segment 648518346349506159 vertex labels regenerated in: 4.1778717041015625 seconds.
648518346349506617
Segment 648518346349506617 vertex labels regenerated in: 4.22973895072937 seconds.
648518346349506770
Segment 648518346349506770 vertex labels regenerated in: 6.733789920806885 seconds.
648518346349506952
Segment 648518346349506952 vertex labels regenerated in: 2.44861102104187 seconds.
648518346349507087
Segment 648518346349507087 vertex labels regenerated in: 4.795411825180054

In [5]:
#ta3p100.CoarseLabel()

In [6]:
#(schema.jobs & "table_name='__coarse_label'").delete()